# pvtools Validation

The objective of this notebook if to validate the pvtools lift-over method using database lookups. 

# Possible issues with pvtools to evaluate

1. LiftOver: positions inserted/deleted in one reference version vs. another can alter position relative to start in different references. This is generally adressed with a chain file that defines these areas and allows adjustment. 
2. +/- strand 

# Methods

This will focus on the accuracy of the positions and alleles reported for pvtools, using data from NCBI and Ensembl to compare database lookups with lifted-over values for positions. 
Additionally, the accuracy of reported ref and alt alleles will be compared, since these might differ. 

Data will be stored in an untracked directory called "valid_data"
The following cell will populate with required resource files. 

In [1]:
!mkdir valid_data
!curl https://ftp.ncbi.nih.gov/refseq/H_sapiens/RefSeqGene/Aligned2RefSeqGene --output valid_data/NG_ACCESSIONS.txt
!curl https://ftp.ncbi.nih.gov/refseq/H_sapiens/RefSeqGene/GCF_000001405.25_refseqgene_alignments.gff3 --output valid_data/NG_ALIGNMENTS_38.gff3
!curl https://ftp.ncbi.nih.gov/refseq/H_sapiens/RefSeqGene/gene_RefSeqGene --output valid_data/NM_ACCESSIONS.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  703k  100  703k    0     0  2931k      0 --:--:-- --:--:-- --:--:-- 2931k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2216k  100 2216k    0     0  5864k      0 --:--:-- --:--:-- --:--:-- 5849k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  192k  100  192k    0     0  1375k      0 --:--:-- --:--:-- --:--:-- 1375k


## Ensembl Access

Next we define a function to collect data from Ensembl based on rsid. 

In [1]:
import sys
import json

import requests

import pandas as pd
import numpy as np

GRCH38_SERVER = 'https://rest.ensembl.org'
GRCH37_SERVER = 'https://grch37.rest.ensembl.org'
URL_EXT = '/variation/homo_sapiens'
HEADERS = { 'Content-Type' : 'application/json', 'Accept' : 'application/json'}

def make_request(rsids, server, url_ext, headers):
    var_data = []
    rsids_formatted = '","'.join(rsids)
    r = requests.post(server+url_ext, headers=headers, data=f'{{ "ids" : ["{rsids_formatted}"] }}')
    if not r.ok:
        r.raise_for_status()
        sys.exit()
    decoded = r.json()
    for rsid, data in decoded.items():
        var_data.append(
            {
                "rsid": rsid,
                "reference": data["mappings"][0]["assembly_name"],
                "pos": data["mappings"][0]["start"],
                "ref_allele": data["mappings"][0]["allele_string"].split("/")[0],
                "alt_allele": ",".join(data["mappings"][0]["allele_string"].split("/")[1:])
            }
        )
    return var_data

## Read pvtools output

Need to have df defined and get rsids

In [147]:
pvtools_snp_data = pd.read_csv("SLCO1B1/SNP_Table.tsv", delimiter = "\t", na_values = [".", "??"])
pvtools_snp_data = pvtools_snp_data.dropna(axis = 0, subset = ["rs_ID"])
rsids = pvtools_snp_data.rs_ID.to_numpy()

## Call Ensembl api to get data from rsids

Also convert to df and merge

In [148]:
ens_G38_data = make_request(rsids[rsids != "."], GRCH38_SERVER, URL_EXT, HEADERS)
ens_G37_data = make_request(rsids[rsids != "."], GRCH37_SERVER, URL_EXT, HEADERS)
ens_G38_df = pd.DataFrame(ens_G38_data).rename(axis = 1, mapper = {"pos": "GRCH38_POS", "ref_allele": "GRCH38_REF", "alt_allele": "GRCH38_ALT"}).drop("reference", axis = 1)
ens_G37_df = pd.DataFrame(ens_G37_data).rename(axis = 1, mapper = {"pos": "GRCH37_POS", "ref_allele": "GRCH37_REF", "alt_allele": "GRCH37_ALT"}).drop("reference", axis = 1)
ens_data = ens_G38_df.merge(ens_G37_df, on = "rsid")
ens_data.head()

,rsid,GRCH38_POS,GRCH38_REF,GRCH38_ALT,GRCH37_POS,GRCH37_REF,GRCH37_ALT
0,rs74064260,21239310,G,C,21392244,G,C
1,rs56101265,21172782,T,C,21325716,T,C
2,rs11045819,21176879,C,"A,T",21329813,C,"A,T"
3,rs4149085,21239356,T,C,21392290,T,C
4,rs59113707,21202555,C,G,21355489,C,G


## Subset pvtools and reorganize to work with ensembl output

In [149]:
pvtools_snp_data = pvtools_snp_data[["rs_ID", "GRCh38_Position", "GRCh37_Position", "Reference_Allele", "Alternative_Allele", "Start_Position", "ATG_Position","Transcript_Position"]]
pvtools_snp_data.rename(axis = 1, mapper = {"rs_ID": "rsid", "GRCh38_Position": "GRCH38_POS", "GRCh37_Position": "GRCH37_POS", "Reference_Allele": "REF", "Alternative_Allele": "ALT"}, inplace = True)
pvtools_snp_data.head()

,rsid,GRCH38_POS,GRCH37_POS,REF,ALT,Start_Position,ATG_Position,Transcript_Position
1,rs4149015,21130388.0,21283322.0,G,A,4195.0,-11187.0,c.-910
2,rs397978904,21130465.0,21283399.0,T,G,4272.0,-11110.0,c.-833
3,rs11835045,21130885.0,21283819.0,T,C,4692.0,-10690.0,c.-413
5,rs59710386,21131076.0,21284010.0,A,C,4883.0,-10499.0,c.-222
6,rs139257324,21172734.0,21325668.0,C,T,46541.0,31159.0,c.169


## Merge pvtools data and Ensembl data

In [150]:
compare_snp_data = pvtools_snp_data.merge(ens_data, on = "rsid")
compare_snp_data.head()

,rsid,GRCH38_POS_x,GRCH37_POS_x,REF,ALT,Start_Position,ATG_Position,Transcript_Position,GRCH38_POS_y,GRCH38_REF,GRCH38_ALT,GRCH37_POS_y,GRCH37_REF,GRCH37_ALT
0,rs4149015,21130388.0,21283322.0,G,A,4195.0,-11187.0,c.-910,21130388,G,"A,C,T",21283322,G,"A,C,T"
1,rs397978904,21130465.0,21283399.0,T,G,4272.0,-11110.0,c.-833,21130465,T,G,21283399,T,G
2,rs11835045,21130885.0,21283819.0,T,C,4692.0,-10690.0,c.-413,21130885,T,C,21283819,T,C
3,rs59710386,21131076.0,21284010.0,A,C,4883.0,-10499.0,c.-222,21131076,A,C,21284010,A,C
4,rs139257324,21172734.0,21325668.0,C,T,46541.0,31159.0,c.169,21172734,C,T,21325668,C,T


## Make comparisons

In [151]:
# Where does GRCh38 position mis-match?

compare_snp_data[compare_snp_data["GRCH38_POS_y"] != compare_snp_data["GRCH38_POS_x"]]

,rsid,GRCH38_POS_x,GRCH37_POS_x,REF,ALT,Start_Position,ATG_Position,Transcript_Position,GRCH38_POS_y,GRCH38_REF,GRCH38_ALT,GRCH37_POS_y,GRCH37_REF,GRCH37_ALT


In [152]:
# Where does GRCh37 position mis-match?

compare_snp_data[compare_snp_data["GRCH37_POS_y"] != compare_snp_data["GRCH37_POS_x"]]

,rsid,GRCH38_POS_x,GRCH37_POS_x,REF,ALT,Start_Position,ATG_Position,Transcript_Position,GRCH38_POS_y,GRCH38_REF,GRCH38_ALT,GRCH37_POS_y,GRCH37_REF,GRCH37_ALT


In [153]:
# Where does ref allele mismatch?

compare_snp_data[compare_snp_data["REF"] != compare_snp_data["GRCH38_REF"]]

,rsid,GRCH38_POS_x,GRCH37_POS_x,REF,ALT,Start_Position,ATG_Position,Transcript_Position,GRCH38_POS_y,GRCH38_REF,GRCH38_ALT,GRCH37_POS_y,GRCH37_REF,GRCH37_ALT


In [154]:
# Where does alt allele mis-match?

compare_snp_data[compare_snp_data["ALT"] != compare_snp_data["GRCH38_ALT"]]

,rsid,GRCH38_POS_x,GRCH37_POS_x,REF,ALT,Start_Position,ATG_Position,Transcript_Position,GRCH38_POS_y,GRCH38_REF,GRCH38_ALT,GRCH37_POS_y,GRCH37_REF,GRCH37_ALT
0,rs4149015,21130388.0,21283322.0,G,A,4195.0,-11187.0,c.-910,21130388,G,"A,C,T",21283322,G,"A,C,T"
8,rs2306283,21176804.0,21329738.0,A,G,50611.0,35229.0,c.388,21176804,A,"G,T",21329738,A,"G,T"
11,rs145144129,21176871.0,21329805.0,G,A,50678.0,35296.0,c.455,21176871,G,C,21329805,G,C
12,rs11045819,21176879.0,21329813.0,C,A,50686.0,35304.0,c.463,21176879,C,"A,T",21329813,C,"A,T"
15,rs141467543,21178612.0,21331546.0,A,G,52419.0,37037.0,c.518,21178612,A,"C,G",21331546,A,"C,G"
21,rs2291076,21179053.0,21331987.0,C,T,52860.0,37478.0,c.727+33,21179053,C,"A,T",21331987,C,"A,T"
26,rs55901008,21200595.0,21353529.0,T,C,74402.0,59020.0,c.1058,21200595,T,"A,C",21353529,T,"A,C"
31,rs56387224,21202649.0,21355583.0,A,G,76456.0,61074.0,c.1294,21202649,A,"G,T",21355583,A,"G,T"
33,rs72559748,21205921.0,21358855.0,A,G,79728.0,64346.0,c.1385,21205921,A,"C,G",21358855,A,"C,G"
34,rs74064211,21205988.0,21358922.0,C,T,79795.0,64413.0,c.1452,21205988,C,"G,T",21358922,C,"G,T"


## Generate lifted-over data for NT, NG, NM references

See scripts folder for generation of chain files


In [155]:
with open("variants_grch38.bed", "w") as bed_38:
    for i, r in compare_snp_data.iterrows():
        bed_38.write(f"chr12\t{r['GRCH38_POS_y'] - 1}\t{r['GRCH38_POS_y']}\t{r['rsid']}\n")

In [156]:
!scripts/ucsc_tools/liftOver variants_grch38.bed scripts/chr12.fa-NG_011745_1.fa.chain variants_NG_011745_1.bed variants_NG_011745_1.failed.txt

Reading liftover chains
Mapping coordinates


In [157]:
!scripts/ucsc_tools/liftOver variants_grch38.bed scripts/chr12.fa-NM_006446_5.fa.chain variants_NM_006446_5.bed variants_NM_006446_5.failed.bed

Reading liftover chains
Mapping coordinates


In [162]:
with open("variants_NG_011745_1.bed", "r") as ng_in:
    ng_vars = [l.strip().split("\t") for l in ng_in.readlines()]
with open("variants_NM_006446_5.bed", "r") as nm_in:
    nm_vars = [l.strip().split("\t") for l in nm_in.readlines()]
ng = [{"rsid": i[3], "NG_position": int(i[2])} for i in ng_vars]
nm = [{"rsid": i[3], "NM_position": int(i[2]) - 104} for i in nm_vars]
ng_df = pd.DataFrame(ng)
nm_df = pd.DataFrame(nm)
n_df = ng_df.merge(nm_df, on = "rsid")

In [163]:
compare_snp_data = compare_snp_data.merge(n_df, on = "rsid")

In [1]:
compare_snp_data[compare_snp_data["Start_Position"] != compare_snp_data["NG_position"]]

NameError: name 'compare_snp_data' is not defined

In [165]:
compare_snp_data

,rsid,GRCH38_POS_x,GRCH37_POS_x,REF,ALT,Start_Position,ATG_Position,Transcript_Position,GRCH38_POS_y,GRCH38_REF,GRCH38_ALT,GRCH37_POS_y,GRCH37_REF,GRCH37_ALT,NG_position_x,NM_position_x,NG_position_y,NM_position_y
0,rs139257324,21172734.0,21325668.0,C,T,46541.0,31159.0,c.169,21172734,C,T,21325668,C,T,46541,273,46541,169
1,rs373327528,21172776.0,21325710.0,G,A,46583.0,31201.0,c.211,21172776,G,A,21325710,G,A,46583,315,46583,211
2,rs56101265,21172782.0,21325716.0,T,C,46589.0,31207.0,c.217,21172782,T,C,21325716,T,C,46589,321,46589,217
3,rs56061388,21174595.0,21327529.0,T,C,48402.0,33020.0,c.245,21174595,T,C,21327529,T,C,48402,349,48402,245
4,rs2306283,21176804.0,21329738.0,A,G,50611.0,35229.0,c.388,21176804,A,"G,T",21329738,A,"G,T",50611,492,50611,388
5,rs11045818,21176827.0,21329761.0,G,A,50634.0,35252.0,c.411,21176827,G,A,21329761,G,A,50634,515,50634,411
6,rs2306282,21176868.0,21329802.0,A,G,50675.0,35293.0,c.452,21176868,A,G,21329802,A,G,50675,556,50675,452
7,rs145144129,21176871.0,21329805.0,G,A,50678.0,35296.0,c.455,21176871,G,C,21329805,G,C,50678,559,50678,455
8,rs11045819,21176879.0,21329813.0,C,A,50686.0,35304.0,c.463,21176879,C,"A,T",21329813,C,"A,T",50686,567,50686,463
9,rs72559745,21176883.0,21329817.0,A,G,50690.0,35308.0,c.467,21176883,A,G,21329817,A,G,50690,571,50690,467
